In [21]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from prettytable import PrettyTable

In [23]:
bdif = pd.read_excel('bdif_pre.xlsx')
bdif.head()

,parte,año,cod_com,cod_prov,cod_mun,probignicion,idcausa,idmotivacion,idinvestigacioncausa,idcertidumbrecausa,...,x_lon,y_lat,cuadricula,hoja,mapa_militar_cod,superficiearboladatotal,superficienoarboladatotal,superficienoarboladaagricola,superficienoarboladaotras,areaquemada
0,1974020249,1974,11,NaN,NaN,0.0,290,NaN,NaN,2,...,-2.370578,38.350818,D08,605.0,0605D08,8.0,0.0,0.0,0.0,8.0
1,1974020374,1974,11,NaN,NaN,0.0,240,NaN,NaN,2,...,-1.795473,38.526550,B01,705.0,0705B01,0.2,0.1,0.0,0.0,0.3
2,1974020459,1974,11,NaN,NaN,0.0,240,NaN,NaN,2,...,-0.980171,38.965835,L08,704.0,0704L08,0.6,0.0,0.0,0.0,0.6
3,1974022274,1974,11,NaN,NaN,0.0,250,NaN,NaN,2,...,-2.369011,38.531046,B08,605.0,0605B08,2.0,0.0,0.0,0.0,2.0
4,1974022457,1974,11,NaN,NaN,0.0,500,NaN,NaN,2,...,-2.139581,38.529585,B10,605.0,0605B10,0.5,0.0,0.0,0.0,0.5


In [24]:
# Copiamos las coordenadas de los municipios
bdif['lon'] = bdif['lon_ETRS89']
bdif['lat'] = bdif['lat_ETRS89']

In [25]:
# Necesitamos cargar el dataset del NGMEP
municipios = pd.read_excel('municipiosNGMEP.xlsx')
municipios.head()

,COD_INE,ID_REL,COD_GEO,COD_PROV,PROVINCIA,NOMBRE_ACTUAL,POBLACION_MUNI,SUPERFICIE,PERIMETRO,COD_INE_CAPITAL,CAPITAL,POBLACION_CAPITAL,HOJA_MTN25_ETRS89,LONGITUD_ETRS89,LATITUD_ETRS89,ORIGENCOOR,ALTITUD,ORIGENALTITUD
0,1001000000,1010014,1010,1,Araba/Álava,Alegría-Dulantzi,2975,1994.5872,35069,1001000101,Alegría-Dulantzi,2860,0113-3,-2.512437,42.839812,Mapa,568.0,MDT
1,1002000000,1010029,1020,1,Araba/Álava,Amurrio,10313,9629.6800,65381,1002000201,Amurrio,9238,0086-4,-3.000073,43.054278,Mapa,219.0,MDT
2,1003000000,1010035,1030,1,Araba/Álava,Aramaio,1409,7308.9600,42097,1003000601,Ibarra,758,0087-4,-2.565400,43.051197,Mapa,333.0,MDT
3,1004000000,1010040,1040,1,Araba/Álava,Artziniega,1832,2728.7300,22886,1004000101,Artziniega,1697,0086-1,-3.127917,43.120844,Mapa,210.0,MDT
4,1006000000,1010066,1060,1,Araba/Álava,Armiñón,232,1297.2700,24707,1006000101,Armiñón,113,0137-4,-2.871835,42.723262,Mapa,467.0,MDT


In [26]:
# Para los registros que no tengan coordenadas buscaremos los mas cercanos
# Guardamos las coordenadas de los municipios y construimos el árbol
coordenadas_municipios = municipios.dropna(subset=['LONGITUD_ETRS89', 'LATITUD_ETRS89'])
tree = cKDTree(coordenadas_municipios[['LONGITUD_ETRS89', 'LATITUD_ETRS89']])

In [28]:
# Extraemos los registros que no tienen coordenadas
bdif_faltantes = bdif[bdif['lon'].isna() & bdif['lat'].isna()]

In [29]:
# Para las filas sin lon y lat, buscar el más cercano
for indice, fila in bdif_faltantes.iterrows():
    if not pd.isna(fila['x_lon']) and not pd.isna(fila['y_lat']):
        
        # Buscamos las coordenadas más cercanas
        distancia, ind = tree.query([fila['x_lon'], fila['y_lat']])

        # Tomamos los datos del municipio más cercano
        cercano = coordenadas_municipios.iloc[ind]

        # Imputa los valores en bdif
        bdif.at[indice, 'lon'] = cercano['LONGITUD_ETRS89']
        bdif.at[indice, 'lat'] = cercano['LATITUD_ETRS89']
        bdif.at[indice, 'cod_ine'] = cercano['COD_INE']
        bdif.at[indice, 'cod_prov'] = cercano['COD_PROV']
        bdif.at[indice, 'id_rel'] = cercano['ID_REL']
        bdif.at[indice, 'provincia'] = cercano['PROVINCIA']
        bdif.at[indice, 'nombre'] = cercano['NOMBRE_ACTUAL']
        bdif.at[indice, 'superficie'] = cercano['SUPERFICIE']
        bdif.at[indice, 'altitud'] = cercano['ALTITUD']
        bdif.at[indice, 'poblacion'] = cercano['POBLACION_MUNI']

In [36]:
bdif.columns

Index(['parte', 'año', 'cod_com', 'cod_prov', 'cod_mun', 'probignicion',
       'idcausa', 'idmotivacion', 'idinvestigacioncausa', 'idcertidumbrecausa',
       'idcausante', 'diastormenta', 'diasultimalluvia', 'tempmaxima',
       'humrelativa', 'velocidadviento', 'direccionviento',
       'primeranotificaciondesde112', 'detectadopor', 'nummunicipiosafectados',
       'huso_incendio', 'x_incendio', 'y_incendio', 'iddatum', 'deteccion',
       'extinguido', 'duracion', 'combustible', 'tipofuego', 'lugar', 'mes',
       'tipodia', 'id_rel', 'dc', 'cod_ine', 'provincia', 'nombre',
       'poblacion', 'superficie', 'lon_ETRS89', 'lat_ETRS89', 'lon', 'lat',
       'altitud', 'x_lon', 'y_lat', 'cuadricula', 'hoja', 'mapa_militar_cod',
       'superficiearboladatotal', 'superficienoarboladatotal',
       'superficienoarboladaagricola', 'superficienoarboladaotras',
       'areaquemada'],
      dtype='object')

In [40]:
columnas = ['parte','año','cod_com','cod_prov','probignicion','diastormenta','diasultimalluvia','tempmaxima',
            'humrelativa','velocidadviento','direccionviento','detectadopor','deteccion','combustible','tipofuego',
            'lugar','mes','tipodia','id_rel','provincia','poblacion','superficie','altitud','areaquemada','lon','lat']
bdif = bdif[columnas]
# Exportar a un nuevo archivo
bdif[columnas].to_excel('bdif_geograficas.xlsx', index=False)

### Tabla de resultados

In [41]:
# Obtener dimensiones de la base de datos
filas, columnas = bdif.shape

# Creamos la tabla
tabla = PrettyTable(['Variable', 'No nulos', 'Tipo'])

# Añadimos los datos
for columna in bdif.columns:
    no_nulos = bdif[columna].notnull().sum()
    tipo = bdif[columna].dtype
    tabla.add_row([columna, no_nulos, tipo])

# Info de las dimensiones del base de datos
tabla.add_row(['DIMENSIONES', f'{filas} filas', f'{columnas} columnas'])

# Mostrar la tabla
print(tabla)

+------------------+--------------+----------------+
|     Variable     |   No nulos   |      Tipo      |
+------------------+--------------+----------------+
|      parte       |    588555    |     int64      |
|       año        |    588555    |     int64      |
|     cod_com      |    588555    |     int64      |
|     cod_prov     |    588555    |    float64     |
|   probignicion   |    475296    |    float64     |
|   diastormenta   |    24315     |    float64     |
| diasultimalluvia |    223547    |    float64     |
|    tempmaxima    |    296771    |    float64     |
|   humrelativa    |    291606    |    float64     |
| velocidadviento  |    276913    |    float64     |
| direccionviento  |    218404    |    float64     |
|   detectadopor   |    588055    |     object     |
|    deteccion     |    588555    | datetime64[ns] |
|   combustible    |    588555    |     object     |
|    tipofuego     |    588554    |     object     |
|      lugar       |    588552    |     object